In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D,Input
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.models import Model

Using TensorFlow backend.


In [2]:
def get_generator():

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense((28*28*1), activation='tanh'))
        model.add(Reshape((28,28,1)))

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)


In [3]:
def get_discriminator():
        
        model = Sequential()

        model.add(Flatten(input_shape=(28,28,1)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=(28,28,1))
        validity = model(img)

        return Model(img, validity)

In [4]:
from keras.datasets import mnist
import numpy as np

In [5]:
(X_train, _), (_, _) = mnist.load_data()

In [6]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

X_train = np.expand_dims(X_train, axis=3)


In [7]:
discriminator = get_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [8]:
generator = get_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [9]:
# noise fed to generator 
z = Input(shape=(100,))
img = generator(z)

In [10]:
discriminator.trainable = False


In [11]:
valid = discriminator(img)


In [12]:
combined = Model(z, valid)

In [13]:
combined.compile(loss='binary_crossentropy',  optimizer=Adam(0.0002, 0.5))

In [14]:
noise = np.random.normal(0, 1, (32, 100))
gen_imgs = generator.predict(noise)

In [15]:
gen_imgs.shape

(32, 28, 28, 1)

In [16]:
idx = np.random.randint(0, X_train.shape[0], 32)
imgs = X_train[idx]

In [17]:
imgs.shape

(32, 28, 28, 1)

In [18]:
from tqdm import tqdm
import matplotlib.pyplot as plt

In [24]:
def save_imgs(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = generator.predict(noise)
#         pred = discriminator.predict()
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("mnist_%d.png" % epoch)
        plt.close()

In [26]:
for epoch in tqdm(range(10000)):
    idx = np.random.randint(0, X_train.shape[0], 32)
    imgs = X_train[idx]
    
    noise = np.random.normal(0, 1, (32, 100))
    
    gen_imgs = generator.predict(noise)
    
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((32, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((32, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (128, 100))
    valid_y = np.array([1] * 128)

    g_loss = combined.train_on_batch(noise, valid_y)
    if epoch % 10 == 0:
                save_imgs(epoch)
            
#     if epoch % 10000 == 0:
        
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss, 100*d_loss, g_loss))
        

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]C:\Users\spari\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.696828, acc.: 69.68%] [G loss: 0.786933]


  0%|                                                                                | 1/10000 [00:00<56:28,  2.95it/s]

1 [D loss: 0.694316, acc.: 69.43%] [G loss: 0.762952]
2 [D loss: 0.725981, acc.: 72.60%] [G loss: 0.801412]
3 [D loss: 0.691788, acc.: 69.18%] [G loss: 0.784234]
4 [D loss: 0.704810, acc.: 70.48%] [G loss: 0.797652]


  0%|                                                                                | 5/10000 [00:00<14:45, 11.29it/s]

...

100%|████████████████████████████████████████████████████████████████████████████▉| 9991/10000 [08:33<00:00, 19.46it/s]

9991 [D loss: 0.696688, acc.: 69.67%] [G loss: 0.759941]
9992 [D loss: 0.691628, acc.: 69.16%] [G loss: 0.749310]
9993 [D loss: 0.743034, acc.: 74.30%] [G loss: 0.753848]
9994 [D loss: 0.679658, acc.: 67.97%] [G loss: 0.742177]
9995 [D loss: 0.699118, acc.: 69.91%] [G loss: 0.750753]


100%|████████████████████████████████████████████████████████████████████████████▉| 9996/10000 [08:33<00:00, 19.46it/s]

9996 [D loss: 0.719710, acc.: 71.97%] [G loss: 0.766018]
9997 [D loss: 0.703235, acc.: 70.32%] [G loss: 0.767110]
9998 [D loss: 0.687705, acc.: 68.77%] [G loss: 0.754628]
9999 [D loss: 0.664877, acc.: 66.49%] [G loss: 0.757794]


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [08:33<00:00, 19.47it/s]


In [ ]:
# generator = generator()
# generator.predict(noise)